In [1]:
import sys
import scib
from sklearn.metrics.pairwise import cosine_similarity
import scanpy
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
adata_beta = scanpy.read_h5ad('../data/beta_run_2.h5ad')
adata_ref = scanpy.read_h5ad('data/mouse_atlas_with_embryonic.h5ad')

Couldn't save scMMGAN values in anndata object due to size

In [3]:
out = np.load(f'results/output_mouse_normal_to_all_cycle_1_correspondence_15_corr_correspondence_all_training_counter_normal_t2d_filtered.npz')
ctrl_magan = pd.DataFrame(out['arr_0'], index=adata_ref.obs_names, columns=adata_beta.var_names)

In [4]:
adata_post_corr = scanpy.AnnData(ctrl_magan)
adata_post_all = scanpy.concat((adata_post_corr, adata_beta), label='batch')

In [5]:
adata_post_all.obsm['X_pc'] = np.vstack((adata_ref.obsm['X_scmmgan_pc'], adata_beta.obsm['X_pca']))

In [6]:
evaluation_summary = pd.DataFrame(columns=['pre_post_corr'])
adata_post_all.obs['cell_type'] = 'beta cell'
evaluation_summary.loc['Batch ASW'] = scib.me.silhouette_batch(adata_post_all, batch_key='batch', label_key='cell_type', embed='X_pc', )

mean silhouette per group:            silhouette_score
group                      
beta cell          0.977466


In [7]:
intersection = list(set(adata_ref.var_names).intersection(adata_post_corr.var_names))
print(len(intersection))
ref_x = adata_ref[:, intersection].X.toarray()
post_x = adata_post_corr[:, intersection].X.toarray()

pre_post_corr = []
for i in tqdm(range(adata_ref.n_obs)):
    pre_post_corr.append(cosine_similarity(ref_x[i].reshape(1,-1), post_x[i].reshape(1,-1))[0][0])

16325


100%|██████████| 85129/85129 [00:35<00:00, 2365.38it/s]


In [8]:
evaluation_summary.loc['Mean cell cosine similarity pre- and post-alignment'] = np.mean(pre_post_corr)

In [9]:
adata_ref.obs['pre_post_corr'] = pre_post_corr
evaluation_summary = pd.concat((evaluation_summary, adata_ref.obs[['AT_committment', 'pre_post_corr']].groupby('AT_committment').mean()))
evaluation_summary = pd.concat((evaluation_summary, adata_ref.obs[['disease', 'pre_post_corr']].groupby('disease').mean()))
evaluation_summary = pd.concat((evaluation_summary, adata_ref.obs[['development_stage', 'pre_post_corr']].groupby('development_stage').mean()))
evaluation_summary = pd.concat((evaluation_summary, adata_ref.obs[['dataset', 'pre_post_corr']].groupby('dataset').mean()))
evaluation_summary.to_csv('results/scMMGAN_evaluation_summary.csv')

In [10]:
evaluation_summary

,pre_post_corr
Batch ASW,0.977466
Mean cell cosine similarity pre- and post-alignment,0.575798
HFD AT 1,0.525689
HFD AT 2,0.580180
HFD AT 3,0.600564
HFD AT 4,0.629151
OB/OB AT 1,0.545472
OB/OB AT 2,0.669184
OB/OB AT 3,0.589378
Uncommitted,0.584331
